# Set-up

In [1]:
# DATA MANIPULATION
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random # generating random numbers
import datetime # manipulating date formats
from operator import add # elementwise addition

# VIZUALIZATION
import matplotlib.pyplot as plt # basic plotting
import seaborn # for prettier plots
#import folium # plotting data on interactive maps
%matplotlib inline

# SUPERVISED LEARNING
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

# Reducing train data

Try reducing the dataset size by choosing a subset of stores/items.

In [2]:
dtypes = {'id':'int64', 'item_nbr':'int32', 'store_nbr':'int8', 'onpromotion':str}

if False:    
    # Reading train data
    train = pd.read_csv('../input/train.csv', dtype=dtypes, parse_dates=['date'])

    # Filter out some stores
    sales = pd.read_csv('../input/processed/sales+.csv', parse_dates=['date'])
    u_stores = sales.store_nbr.unique()
    random.seed(115599)
    random_stores = sorted(u_stores[random.sample(range(len(u_stores)), 10) ])

    train = train[lambda df: (df.store_nbr.isin(random_stores)) & (df.date>="2015-01-01")]
    train.to_csv('../input/processed/train_min.csv',index=False)

# Load datasets

In [3]:
# Reading train
train = pd.read_csv('../input/processed/train_min.csv', dtype=dtypes, parse_dates=['date'])

# Reading (pre-processed) sales 
sales = pd.read_csv('../input/processed/sales+.csv', parse_dates=['date'])

# Reading items
items = pd.read_csv('../input/items.csv')

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16217115 entries, 0 to 16217114
Data columns (total 6 columns):
id             int64
date           datetime64[ns]
store_nbr      int8
item_nbr       int32
unit_sales     float64
onpromotion    object
dtypes: datetime64[ns](1), float64(1), int32(1), int64(1), int8(1), object(1)
memory usage: 572.2+ MB


In [5]:
train.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,38594264,2015-01-01,25,103665,12.0,False
1,38594265,2015-01-01,25,105575,23.0,False
2,38594266,2015-01-01,25,108634,1.0,False
3,38594267,2015-01-01,25,108698,6.0,False
4,38594268,2015-01-01,25,108786,6.0,False


In [6]:
sales.head()

,date,store_nbr,transactions,city,state,type,cluster,dcoilwtico,year,month,...,state_Santa Elena,state_Santo Domingo de los Tsachilas,state_Tungurahua,type_A,type_B,type_C,type_D,type_E,lag_7,lag_annual
0,2014-01-02,1,1875.0,Quito,Pichincha,D,13.0,95.14,2014.0,1.0,...,-0.140447,-0.227312,-0.204583,-0.43985,-0.401878,-0.635502,1.387063,-0.272575,7.591357,7.655391
1,2014-01-03,1,1817.0,Quito,Pichincha,D,13.0,93.66,2014.0,1.0,...,-0.140447,-0.227312,-0.204583,-0.43985,-0.401878,-0.635502,1.387063,-0.272575,7.612337,7.514255
2,2014-01-06,1,1754.0,Quito,Pichincha,D,13.0,93.12,2014.0,1.0,...,-0.140447,-0.227312,-0.204583,-0.43985,-0.401878,-0.635502,1.387063,-0.272575,7.236339,6.255750
3,2014-01-07,1,1713.0,Quito,Pichincha,D,13.0,93.31,2014.0,1.0,...,-0.140447,-0.227312,-0.204583,-0.43985,-0.401878,-0.635502,1.387063,-0.272575,7.008505,7.499977
4,2014-01-09,1,1855.0,Quito,Pichincha,D,13.0,91.36,2014.0,1.0,...,-0.140447,-0.227312,-0.204583,-0.43985,-0.401878,-0.635502,1.387063,-0.272575,7.536897,7.555382


In [7]:
items.head()

,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1


# Dataset preprocessing

In [8]:
drop_cols = ['city','state','type','cluster', 'transactions2', 'transactions3'] + \
            [c for c in sales.columns if c.startswith("state_") or c.startswith("type_") or c.startswith("city_")]
        
sales.drop(drop_cols,axis=1,inplace=True)
#sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45373 entries, 0 to 45372
Data columns (total 67 columns):
date            45373 non-null datetime64[ns]
store_nbr       45373 non-null int64
transactions    45373 non-null float64
dcoilwtico      45373 non-null float64
year            45373 non-null float64
month           45373 non-null float64
day             45373 non-null float64
week            45373 non-null float64
dow             45373 non-null float64
dayofyear       45373 non-null float64
dayoff          45373 non-null bool
store_nbr_1     45373 non-null float64
store_nbr_2     45373 non-null float64
store_nbr_3     45373 non-null float64
store_nbr_4     45373 non-null float64
store_nbr_5     45373 non-null float64
store_nbr_6     45373 non-null float64
store_nbr_7     45373 non-null float64
store_nbr_8     45373 non-null float64
store_nbr_9     45373 non-null float64
store_nbr_10    45373 non-null float64
store_nbr_11    45373 non-null float64
store_nbr_12    45373 non-null 

In [9]:
train = pd.merge(train, items, how='left', on=['item_nbr'])
train = pd.merge(train, sales, how='left', on=['date','store_nbr'])

In [17]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16217115 entries, 0 to 16217114
Data columns (total 74 columns):
id              int64
date            datetime64[ns]
store_nbr       int8
item_nbr        int32
unit_sales      float64
onpromotion     object
family          object
class           int64
perishable      int64
transactions    float64
dcoilwtico      float64
year            float64
month           float64
day             float64
week            float64
dow             float64
dayofyear       float64
dayoff          object
store_nbr_1     float64
store_nbr_2     float64
store_nbr_3     float64
store_nbr_4     float64
store_nbr_5     float64
store_nbr_6     float64
store_nbr_7     float64
store_nbr_8     float64
store_nbr_9     float64
store_nbr_10    float64
store_nbr_11    float64
store_nbr_12    float64
store_nbr_13    float64
store_nbr_14    float64
store_nbr_15    float64
store_nbr_16    float64
store_nbr_17    float64
store_nbr_18    float64
store_nbr_19    float64
stor

In [19]:
# Drop one-hot encoding field
sc = [c for c in train.columns if c.startswith('store_nbr_')]
train.drop(sc,axis=1,inplace=True)
train.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,transactions,dcoilwtico,year,month,day,week,dow,dayofyear,dayoff,lag_7,lag_annual
0,38594264,2015-01-01,25,103665,12.0,False,BREAD/BAKERY,2712,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,38594265,2015-01-01,25,105575,23.0,False,GROCERY I,1045,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,38594266,2015-01-01,25,108634,1.0,False,GROCERY I,1075,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,38594267,2015-01-01,25,108698,6.0,False,DELI,2644,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,38594268,2015-01-01,25,108786,6.0,False,CLEANING,3044,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
